In [10]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from bson import json_util

# Verbindung zur MongoDB herstellen
client = pymongo.MongoClient("mongodb://mongo:27017/")
db = client["car_crashes"]  # Datenbankname
collection = db["crashes"]  # Sammlungname

# Daten abfragen und in DataFrame umwandeln
documents = []
for document in collection.find():
    try:
        # BSON-Dokument in JSON umwandeln und zur Liste hinzufügen
        json_doc = json_util.dumps(document)
        documents.append(json_doc)
    except Exception as e:
        print(f"Fehler beim Verarbeiten eines Dokuments: {e}")

# DataFrame erstellen
df = pd.read_json(json_util.dumps(documents))

# crashDate in datetime umwandeln
df['crashDate'] = pd.to_datetime(df['crashDate'], format='%d.%m.%Y')

# crashTime zu datetime.time konvertieren
df['crashTime'] = pd.to_datetime(df['crashTime'], format='%H:%M').dt.time

# Analyse der Unfallhäufigkeit nach Datum
crash_count_by_date = df['crashDate'].value_counts().sort_index()

# Analyse der Unfallhäufigkeit nach Stunde
df['crashHour'] = pd.to_datetime(df['crashTime'], format='%H:%M').dt.hour
crash_count_by_hour = df['crashHour'].value_counts().sort_index()




InvalidBSON: 'utf-8' codec can't decode byte 0xfd in position 0: invalid start byte

In [ ]:
print("Visualisierung der Unfallhäufigkeit nach Datum")

plt.figure(figsize=(12, 6))
plt.plot(crash_count_by_date.index, crash_count_by_date.values, marker='o')
plt.title('Unfallhäufigkeit nach Datum')
plt.xlabel('Datum')
plt.ylabel('Anzahl Unfälle')
plt.grid(True)
plt.show()

In [ ]:
# Visualisierung der Unfallhäufigkeit nach Stunde
plt.figure(figsize=(12, 6))
plt.bar(crash_count_by_hour.index, crash_count_by_hour.values)
plt.title('Unfallhäufigkeit nach Stunde')
plt.xlabel('Stunde des Tages')
plt.ylabel('Anzahl Unfälle')
plt.grid(True)
plt.show()

In [ ]:
# Analyse der Unfallhäufigkeit nach Ort (hier nach onStreetName)
crash_count_by_street = df['onStreetName'].value_counts()

# Visualisierung der Unfallhäufigkeit nach Ort
plt.figure(figsize=(12, 6))
crash_count_by_street.head(10).plot(kind='bar')
plt.title('Top 10 Straßen mit den meisten Unfällen')
plt.xlabel('Straße')
plt.ylabel('Anzahl Unfälle')
plt.grid(True)
plt.show()
